<a href="https://colab.research.google.com/github/spazewalker/FER_Doggomaniacs/blob/master/Video_emotion_Recognistion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import itertools 
import keras
from keras import layers
from keras.layers import Conv2D, Input, Dense, MaxPool2D,BatchNormalization,Flatten,ZeroPadding2D,Activation,Dropout,LSTM
from keras.models import Model
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
%matplotlib inline

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In classifier model I have removed the Sotftmax Layer and added a Dense layer to give embedding of length 128.

In [ ]:
def classifierModel(X_input):
  '''
  Layer 1
  '''
  X = Conv2D(64,(7,7),strides=(2,2),name = 'conv2')(X_input)
  X = Activation('relu')(X)
  X = MaxPool2D((3,3),strides=(2,2),name='max_pool2')(X)
  X = BatchNormalization()(X)

  '''
  Layer 2
  '''
  X1 = Conv2D(96,(1,1),name='conv4')(X)
  X1 = Activation('relu')(X1)
  X2 = MaxPool2D((3,3),strides=(1,1),name='max_pool3')(X)
  X3 = Conv2D(208,(3,3),name='conv5')(X1)
  X3 = Activation('relu')(X3)
  X4 = Conv2D(64,(1,1),name='conv6')(X2)
  X4 = Activation('relu')(X4)
  chunk_1 = keras.layers.concatenate([X3,X4],axis=3)
  '''
  Layer 3
  '''
  X5 = Conv2D(96,(1,1),name='conv7')(chunk_1)
  X5 = Activation('relu')(X5)
  X6 = MaxPool2D((3,3),strides=(1,1),name='max_pool4')(chunk_1)
  X7 = Conv2D(208, (3,3),name='conv8')(X5)
  X7 = Activation('relu')(X7)
  X8 = Conv2D(64,(1,1),name='conv9')(X6)
  X8 = Activation('relu')(X8)
  chunk_2 = keras.layers.concatenate([X7,X8],axis=3)
  '''
  Layer 4
  '''
  out = Flatten()(chunk_2)
  out = Dropout(0.5)(out)
  out = Dense(128,activation = 'linear')
  return out


1. Clip size will be fixed i.e. the number of images during transition of emotion.
2. s0 and c0 are the initial hidden state and cell state for LSTM
3. Outputs will be an array storing the output of each LSTM cell 
4. Then we will take the last output of LSTM cell as our final output 

In [ ]:
def LSTM_model(clip_size,hidden_state_size,input_size):
  X_input = Input(shape = (clip_size,input_size))
  s0 = Input(shape = (hidden_state_size,))
  c0 = Input(shape = (hidden_state_size,))
  s,c = s0,c0
  outputs = []
  for i in range(clip_size):
    X = classifierModel(X_input[i])
    s,_,c = LSTM(X,initial_state = [s, c],return_state = True)
    out = Dense(7,activation = 'softmax')
    outputs.append(out)
  model = Model(inputs = [X_input,s0,c0],outputs = outputs[-1])
  return model

Clip size yet to be decided and it will be same as hidden_state_size because there will be equal number of LSTM units as number of images in a clip.


In [ ]:
model1 = LSTM_model(clip_size = 12,hidden_state_size = 12,input_size = (48,48))

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()  

In [ ]:
hidden_state_size = 12
s0 = np.zeros((no._of_clips, hidden_state_size))
c0 = np.zeros((no._of_clips, hidden_state_size))

Input_frames are of shape (m,clip_size,input_size)
where input_size for eg. for FER2013 is (48,48)

In [ ]:
model.fit([Input_frames,s0,c0],outputs,epoch=5,batch_size = 128)

In [ ]:

#kf = KFold(n_splits=10)
#kf.get_n_splits(x)
#print(kf)
#model2 = classifierModel((48,48,1))
#model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#for train_index,test_index in kf.split(x,np.asarray(data['emotion'])):

 # X_train, X_test = x[train_index], x[test_index]
 # y_train, y_test = y[train_index], y[test_index]
 # X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
 # X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

 # model2.fit(X_train,y_train,epochs=5,batch_size=128,validation_data=(X_test,y_test))

In [ ]:
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
def plot_confusion_matrix(
        cm,
        classes,
        normalize=False,
        title='Confusion matrix',
        cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=12)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45,color = 'black')
    plt.yticks(tick_marks, classes,color = 'black')

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=12,color='black')
    plt.xlabel('Predicted label', fontsize=12,color='black')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')

In [ ]:
ans = np.argmax(model2.predict(X_test),axis=-1)
y_test = np.argmax(y_test,axis=1)

In [ ]:
matrix = confusion_matrix(y_test,ans)
plot_confusion_matrix(
        matrix,
        classes=class_names,
        normalize=True,
        title='Confusion Matrix')